In [8]:
import os
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import community.community_louvain as community_louvain

In [9]:
# ==============================
# Vytvorenie priečinka pre obrázky
# ==============================
os.makedirs("img", exist_ok=True)

In [2]:
# ==============================
# 1. Load graph
# ==============================
G = nx.read_edgelist("facebook_combined.txt", nodetype=int)
print("Nodes:", G.number_of_nodes())
print("Edges:", G.number_of_edges())

# Layout pre vizualizáciu
pos = nx.spring_layout(G, seed=42)

Nodes: 4039
Edges: 88234


In [10]:
# ==============================
# 2. Raw graph visualization
# ==============================
plt.figure(figsize=(12, 12))
nx.draw_networkx_nodes(G, pos, node_size=10)
nx.draw_networkx_edges(G, pos, alpha=0.05)
plt.title("Facebook social network – raw data")
plt.axis("off")
plt.savefig("img/raw_data.png", dpi=300)
plt.close()

In [ ]:
# ==============================
# 3. Modely detekcie komunít
# ==============================
# Louvain
partition_louvain = community_louvain.best_partition(G)

# Label Propagation (LPA)
communities_lpa = list(nx.algorithms.community.label_propagation_communities(G))
partition_lpa = {}
for idx, comm in enumerate(communities_lpa):
    for node in comm:
        partition_lpa[node] = idx

In [11]:
# ==============================
# 4. Funkcia na vizualizáciu a uloženie
# ==============================
def draw_and_save(graph, partition, title, filename, pos):
    plt.figure(figsize=(12, 12))
    communities = list(set(partition.values()))
    cmap = cm.get_cmap('tab20', len(communities))

    for i, comm in enumerate(communities):
        nodes = [n for n in graph.nodes() if partition[n] == comm]
        nx.draw_networkx_nodes(
            graph, pos,
            nodelist=nodes,
            node_size=20,
            node_color=[cmap(i)]
        )

    nx.draw_networkx_edges(graph, pos, alpha=0.03)
    plt.title(title)
    plt.axis("off")
    plt.savefig(filename, dpi=300)
    plt.close()

# ==============================
# 5. Uloženie vizualizácií do img/
# ==============================
draw_and_save(G, partition_louvain, "Detekcia komunít – Louvain", "img/louvain.png", pos)
draw_and_save(G, partition_lpa, "Detekcia komunít – Label Propagation", "img/lpa.png", pos)


/tmp/ipykernel_26522/1960375212.py:7: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('tab20', len(communities))


In [6]:
# ==============================
# 6. Vyhodnotenie – modularita a počet komunít
# ==============================
mod_louvain = community_louvain.modularity(partition_louvain, G)
mod_lpa = community_louvain.modularity(partition_lpa, G)

print("Louvain – modularita:", mod_louvain, "| počet komunít:", len(set(partition_louvain.values())))
print("LPA     – modularita:", mod_lpa, "| počet komunít:", len(set(partition_lpa.values())))


Louvain – modularita: 0.8349652587713134 | počet komunít: 16
LPA     – modularita: 0.7368407345348218 | počet komunít: 44
